In [4]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..', '..', 'src'))
from sefl_analysis_tools import AnalysisTools
import sefl_analysis_tools as sat
import pandas as pd
import numpy as np

In [5]:
data = pd.read_csv('All_cohorts_freezing_workbook_10_22_24.csv')
data = data[data['cohort'].isin(['ptsd2', 'ptsd3', 'ptsd4', 'ptsd5', 'ptsd6', 'ptsd9'])]
data_old = pd.read_csv('PTSD_filtered.csv')


In [6]:
merged_data = pd.merge(data_old, data[['id', 'cohort', 'freezing_1_75s', 'day']], on=['id', 'cohort', 'day'], how='left')
columns = ['freezing', 'freezing_1_75s'] + [col for col in merged_data.columns if col not in ['freezing', 'freezing_1_75s']]
merged_data = merged_data[columns]
merged_data = sat.calculate_age_at_sefla(merged_data)
merged_data.head()

/Users/novak/Documents/Columbia/mentored_research/Turi/els git/els_project/notebooks/novak_analyses/../../src/sefl_analysis_tools.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['dob'] = pd.to_datetime(data['dob'])
/Users/novak/Documents/Columbia/mentored_research/Turi/els git/els_project/notebooks/novak_analyses/../../src/sefl_analysis_tools.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['date'] = pd.to_datetime(data['date'])


,freezing,freezing_1_75s,genotype,id,day,condition,sex,cohort,exposure_mins,analyzed_duration,date,time,els,psilocybin,cohort_id,dob,age_at_sefla,young
0,5.03,0.90,wt,81,sefla,control,male,ptsd2,60.0,60.0,2021-04-28,11:59 AM,n,n,ptsd2_81,2021-02-16,10.142857,True
1,0.45,0.00,wt,81,seflb,control,male,ptsd2,3.5,3.5,2021-04-29,4:40 PM,n,n,ptsd2_81,2021-02-16,10.142857,True
2,8.28,0.62,wt,81,recall1,control,male,ptsd2,5.0,5.0,2021-04-30,1:37 PM,n,n,ptsd2_81,2021-02-16,10.142857,True
3,7.12,0.71,wt,81,recall2,control,male,ptsd2,5.0,5.0,2021-05-03,10:17 AM,n,n,ptsd2_81,2021-02-16,10.142857,True
4,7.57,0.00,wt,81,recall3,control,male,ptsd2,5.0,5.0,2021-05-05,10:04 AM,n,n,ptsd2_81,2021-02-16,10.142857,True


In [7]:
tools = AnalysisTools(merged_data)

In [8]:
result = tools.repeated_measures_anova(
    dv='freezing_1_75s',
    within='day',
    between='condition',
    subject='cohort_id'
)
print("Repeated ANOVA Result for conditional comparison:")
print(result['anova_result'])
print(f"Required Sample Size per Group: {result['required_sample_size']:.4f}")

Repeated ANOVA Result for conditional comparison:
        Source            SS  DF1  DF2            MS          F         p-unc  \
0    condition  28072.514464    1   51  28072.514464  50.983136  3.268044e-09   
1          day  34036.575820    6  306   5672.762637  48.703555  8.240173e-42   
2  Interaction  16613.978262    6  306   2768.996377  23.773244  4.663040e-23   

      p-GG-corr       np2       eps sphericity   W-spher       p-spher  
0           NaN  0.499917       NaN        NaN       NaN           NaN  
1  1.339095e-18  0.488484  0.557914      False  0.080079  2.891779e-17  
2           NaN  0.317938       NaN        NaN       NaN           NaN  
Required Sample Size per Group: 5.0912


In [10]:
results2 = tools.repeated_measures_anova(
    dv='freezing_1_75s',
    within='day',
    between='young',
    subject='cohort_id'
)

print("Repeated ANOVA Result for age comparison:")
print(results2['anova_result'])
print(f"Required Sample Size per Group: {results2['required_sample_size']:.4f}")


Repeated ANOVA Result for age comparison:
        Source            SS  DF1  DF2           MS          F         p-unc  \
0        young   8317.720668    1   51  8317.720668   8.867767  4.433238e-03   
1          day  34036.575820    6  306  5672.762637  34.897146  4.593851e-32   
2  Interaction   2513.099008    6  306   418.849835   2.576639  1.890246e-02   

      p-GG-corr       np2       eps sphericity   W-spher       p-spher  
0           NaN  0.148123       NaN        NaN       NaN           NaN  
1  1.339095e-18  0.406267  0.557914      False  0.080079  2.891779e-17  
2           NaN  0.048093       NaN        NaN       NaN           NaN  
Required Sample Size per Group: 23.5701


In [11]:
sefl_merged_data = merged_data[merged_data['condition'] == 'sefl']
control_merged_data = merged_data[merged_data['condition'] == 'control']

tools3 = AnalysisTools(sefl_merged_data)
tools4 = AnalysisTools(control_merged_data)

results3 = tools3.repeated_measures_anova(
    dv='freezing_1_75s',
    within='day',
    between='young',
    subject='cohort_id'
)

print("Repeated ANOVA Result for age comparison in SEFL:")
print(results3['anova_result'])
print(f"Required Sample Size per Group: {results3['required_sample_size']:.4f}")


results4 = tools4.repeated_measures_anova(
    dv='freezing_1_75s',
    within='day',
    between='young',
    subject='cohort_id'
)

print("Repeated ANOVA Result for age comparison in control:")
print(results4['anova_result'])
print(f"Required Sample Size per Group: {results4['required_sample_size']:.4f}")


Repeated ANOVA Result for age comparison in SEFL:
        Source            SS  DF1  DF2           MS          F         p-unc  \
0        young   4534.471251    1   33  4534.471251   6.793020  1.362837e-02   
1          day  49898.724673    6  198  8316.454112  55.244337  1.026984e-39   
2  Interaction   3061.903995    6  198   510.317332   3.389923  3.322625e-03   

      p-GG-corr       np2       eps sphericity   W-spher   p-spher  
0           NaN  0.170709       NaN        NaN       NaN       NaN  
1  7.594717e-24  0.626038  0.596297      False  0.142178  0.000004  
2           NaN  0.093156       NaN        NaN       NaN       NaN  
Required Sample Size per Group: 20.0718
Repeated ANOVA Result for age comparison in control:
        Source          SS  DF1  DF2          MS         F     p-unc  \
0        young  202.708206    1   16  202.708206  2.463624  0.136073   
1          day  751.829410    6   96  125.304902  4.665035  0.000333   
2  Interaction  194.125688    6   96   32.35

In [20]:
young_merged_data = merged_data[merged_data['young'] == 'True']
old_merged_data = merged_data[merged_data['young'] == 'False']

tools5 = AnalysisTools(young_merged_data)
tools6 = AnalysisTools(old_merged_data)

results5 = tools5.repeated_measures_anova(
    dv='freezing_1_75s',
    within='day',
    between='condition',
    subject='cohort_id'
)

print("Repeated ANOVA Result for condition comparison in young adult mice:")
print(results5['anova_result'])
print(f"Required Sample Size per Group: {results5['required_sample_size']:.4f}")


results6 = tools6.repeated_measures_anova(
    dv='freezing_1_75s',
    within='day',
    between='condition',
    subject='cohort_id'
)

print("Repeated ANOVA Result for condition comparison in adult mice:")
print(results6['anova_result'])
print(f"Required Sample Size per Group: {results6['required_sample_size']:.4f}")


Repeated ANOVA Result for condition comparison in young adult mice:
        Source            SS  DF1  DF2            MS          F         p-unc  \
0    condition  16588.291451    1   37  16588.291451  48.308119  3.300919e-08   
1          day  23739.712533    6  222   3956.618755  51.512615  1.947470e-39   
2  Interaction  14544.492948    6  222   2424.082158  31.559980  2.529499e-27   

      p-GG-corr       np2       eps sphericity   W-spher       p-spher  
0           NaN  0.566278       NaN        NaN       NaN           NaN  
1  2.405788e-13  0.581980  0.478449      False  0.043637  1.022021e-14  
2           NaN  0.460327       NaN        NaN       NaN           NaN  
Required Sample Size per Group: 4.2202
Repeated ANOVA Result for condition comparison in adult mice:
        Source            SS  DF1  DF2           MS          F         p-unc  \
0    condition   7903.681802    1   12  7903.681802   8.914455  1.136455e-02   
1          day  12809.962296    6   72  2134.993716  1

In [17]:
results7 = tools.repeated_measures_anova(
    dv='freezing_1_75s',
    within='day',
    between='sex',
    subject='cohort_id'
)

print("Repeated ANOVA Result for sex comparison:")
print(results7['anova_result'])
print(f"Required Sample Size per Group: {results7['required_sample_size']:.4f}")

Repeated ANOVA Result for sex comparison:
        Source            SS  DF1  DF2           MS          F         p-unc  \
0          sex   1855.188458    1   51  1855.188458   1.742470  1.927205e-01   
1          day  34036.575820    6  306  5672.762637  34.384425  1.128380e-31   
2  Interaction   1771.369628    6  306   295.228271   1.789473  1.008067e-01   

      p-GG-corr       np2       eps sphericity   W-spher       p-spher  
0           NaN  0.033037       NaN        NaN       NaN           NaN  
1  1.339095e-18  0.402701  0.557914      False  0.080079  2.891779e-17  
2           NaN  0.033898       NaN        NaN       NaN           NaN  
Required Sample Size per Group: 115.8316


In [19]:
male_merged_data = merged_data[merged_data['sex'] == 'male']
female_merged_data = merged_data[merged_data['sex'] == 'female']

tools8 = AnalysisTools(male_merged_data)
tools9 = AnalysisTools(female_merged_data)

results8 = tools8.repeated_measures_anova(
    dv='freezing_1_75s',
    within='day',
    between='condition',
    subject='cohort_id'
)

print("Repeated ANOVA Result for condition comparison in male mice:")
print(results8['anova_result'])
print(f"Required Sample Size per Group: {results8['required_sample_size']:.4f}")


results9 = tools9.repeated_measures_anova(
    dv='freezing_1_75s',
    within='day',
    between='condition',
    subject='cohort_id'
)

print("Repeated ANOVA Result for condition comparison in female mice:")
print(results9['anova_result'])
print(f"Required Sample Size per Group: {results9['required_sample_size']:.4f}")


Repeated ANOVA Result for condition comparison in male mice:
        Source            SS  DF1  DF2            MS          F         p-unc  \
0    condition  13205.304329    1   31  13205.304329  37.399790  8.836422e-07   
1          day  20391.657594    6  186   3398.609599  28.485908  4.970885e-24   
2  Interaction   9900.576972    6  186   1650.096162  13.830505  5.614858e-13   

      p-GG-corr       np2       eps sphericity   W-spher       p-spher  
0           NaN  0.546782       NaN        NaN       NaN           NaN  
1  2.148788e-10  0.478868  0.506867      False  0.034375  1.375826e-12  
2           NaN  0.308507       NaN        NaN       NaN           NaN  
Required Sample Size per Group: 4.4522
Repeated ANOVA Result for condition comparison in female mice:
        Source            SS  DF1  DF2            MS          F         p-unc  \
0    condition  15904.070539    1   18  15904.070539  20.097650  2.875598e-04   
1          day  15416.287854    6  108   2569.381309  25.0

In [21]:
results3 = tools3.repeated_measures_anova(
    dv='freezing_1_75s',
    within='day',
    between='sex',
    subject='cohort_id'
)

print("Repeated ANOVA Result for age comparison in SEFL:")
print(results3['anova_result'])
print(f"Required Sample Size per Group: {results3['required_sample_size']:.4f}")


results4 = tools4.repeated_measures_anova(
    dv='freezing_1_75s',
    within='day',
    between='sex',
    subject='cohort_id'
)

print("Repeated ANOVA Result for age comparison in control:")
print(results4['anova_result'])
print(f"Required Sample Size per Group: {results4['required_sample_size']:.4f}")


Repeated ANOVA Result for age comparison in SEFL:
        Source            SS  DF1  DF2           MS          F         p-unc  \
0          sex   2873.783832    1   33  2873.783832   4.003360  5.369122e-02   
1          day  49898.724673    6  198  8316.454112  53.509429  7.161110e-39   
2  Interaction   2095.493223    6  198   349.248871   2.247124  4.035281e-02   

      p-GG-corr       np2       eps sphericity   W-spher   p-spher  
0           NaN  0.108189       NaN        NaN       NaN       NaN  
1  7.594717e-24  0.618539  0.596297      False  0.142178  0.000004  
2           NaN  0.063753       NaN        NaN       NaN       NaN  
Required Sample Size per Group: 33.3375
Repeated ANOVA Result for age comparison in control:
        Source          SS  DF1  DF2          MS         F     p-unc  \
0          sex   18.265030    1   16   18.265030  0.194706  0.664933   
1          day  751.829410    6   96  125.304902  4.793370  0.000257   
2  Interaction  263.164199    6   96   43.86